# Clasificación de Texto con el Dataset TECLA

Este notebook demuestra cómo cargar y utilizar el dataset TECLA de Hugging Face para tareas de clasificación de texto en catalán.

TECLA (TEChniques for LAnguage processing) es un dataset diseñado para tareas de clasificación de texto en catalán.

## Instalación de Dependencias

Antes de comenzar, instale las bibliotecas requeridas ejecutando el siguiente comando en su terminal:

In [1]:
# Ejecuta este comando para instalar todas las dependencias necesarias
# Puedes ejecutarlo directamente en una celda del notebook con el prefijo !

#!pip install -U transformers datasets torch numpy pandas matplotlib seaborn scikit-learn accelerate

# Si prefieres ejecutarlo en tu terminal (sin el prefijo !), usa:
# pip install -U transformers datasets torch numpy pandas matplotlib seaborn scikit-learn accelerate

# Si estás utilizando una GPU y quieres instalar PyTorch con soporte para CUDA:
# Para CUDA 11.8 (ajusta segons la teva versió de CUDA):
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Per a un entorn conda, pots fer servir:
# conda install -c huggingface -c conda-forge transformers datasets pytorch numpy pandas matplotlib seaborn scikit-learn accelerate

print("Ejecuta uno de los comandos anteriores para instalar las dependencias.")
print("Descomenta la línea correspondiente a tu sistema y necesidades.")

Ejecuta uno de los comandos anteriores para instalar las dependencias.
Descomenta la línea correspondiente a tu sistema y necesidades.


In [2]:
# Cargar el dataset de projecte-aina/tecla desde Hugging Face
from datasets import load_dataset

# Cargar el dataset TECLA
tecla_dataset = load_dataset("projecte-aina/tecla")

# Mostrar información sobre el dataset
print(f"Claves del dataset: {tecla_dataset.keys()}")
print(f"Tamaño del conjunto de entrenamiento: {len(tecla_dataset['train'])}")
print(f"Tamaño del conjunto de validación: {len(tecla_dataset['validation'])}")
print(f"Tamaño del conjunto de prueba: {len(tecla_dataset['test'])}")

# Mostrar las columnas disponibles en el dataset
print(f"\nColumnas disponibles: {tecla_dataset['train'].column_names}")

# Mostrar un ejemplo de los datos
print("\nEjemplo de los datos:")
print(tecla_dataset['train'][0])

# Explorar las clases/etiquetas disponibles para label1 y label2
label1_values = set(tecla_dataset['train']['label1'])
label2_values = set(tecla_dataset['train']['label2'])

print(f"\nNúmero de clases en label1: {len(label1_values)}")
print(f"Clases disponibles en label1: {label1_values}")

print(f"\nNúmero de clases en label2: {len(label2_values)}")
print(f"Clases disponibles en label2: {label2_values}")

# Ver la distribución de las etiquetas
print("\nDistribución de label1:")
label1_counts = {}
for label in tecla_dataset['train']['label1']:
    label1_counts[label] = label1_counts.get(label, 0) + 1
for label, count in sorted(label1_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{label}: {count} ({count/len(tecla_dataset['train'])*100:.2f}%)")

print("\nDistribución de label2:")
label2_counts = {}
for label in tecla_dataset['train']['label2']:
    label2_counts[label] = label2_counts.get(label, 0) + 1
for label, count in sorted(label2_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"{label}: {count} ({count/len(tecla_dataset['train'])*100:.2f}%)")

C:\Users\jiahu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Claves del dataset: dict_keys(['train', 'validation', 'test'])
Tamaño del conjunto de entrenamiento: 90700
Tamaño del conjunto de validación: 5669
Tamaño del conjunto de prueba: 17007

Columnas disponibles: ['sentence', 'label1', 'label2']

Ejemplo de los datos:
{'sentence': "L'ACA reactiva el retorn del cànon de l'aigua que s'envia a Tarragona i millorarà l'eficiència del canal de l'esquerra. L'obra corregirà pèrdues d'aigua a la sèquia del Cementiri de Deltebre amb una inversió de 900.000 euros. Després d'alguns exercici de paràlisi, la Comunitat de Regants de l'Esquerra de l'Ebre i l'ACA han signat, aquest dilluns, un nou conveni per reactivar obres de millora de l'eficiència d'infraestructures de reg i evitar la pèrdua d'aigua. Les actuacions començaran al febrer de l'any que ve en un tram de 4,9 quilòmetres de la sèquia del Cementiri, al terme municipal de Deltebre (Baix Ebre).Es col·locaran plaques i llits de graves, làmines de geotèxtil, i es revestiran més de 3 quilòmetres de l

In [3]:
# Importar las bibliotecas esenciales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import torch
from transformers import AutoTokenizer, pipeline

# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Definir el modelo preentrenado para catalán
model_name = "projecte-aina/roberta-base-ca-v2-cased-sts"
print(f"Using model: {model_name}")

# Nos centraremos en label1 para esta tarea de clasificación
# Puedes cambiar a label2 si lo necesitas
target_label = "label1"
print(f"Classification target: {target_label}")

Using device: cpu
Using model: projecte-aina/roberta-base-ca-v2-cased-sts
Classification target: label1


## Enfoque Simplificado con Pipeline

En lugar de realizar un proceso completo de entrenamiento manual, utilizamos directamente la pipeline de Hugging Face para clasificación de texto, lo que simplifica considerablemente el proceso.

In [ ]:
# Enfoque para clasificación de texto usando las categorías específicas de TECLA
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
import numpy as np
from datasets import load_dataset
import torch
import os

# Desactivar TensorFlow para evitar errores
os.environ["USE_TF"] = "0"

try:
    # Preparar el dataset para entrenamiento
    # Vamos a entrenar un modelo específico para label1
    label_column = target_label  # Podemos usar 'label1' o 'label2'
    
    # Obtener etiquetas únicas del dataset
    labels = sorted(list(set(tecla_dataset['train'][label_column])))
    label2id = {label: i for i, label in enumerate(labels)}
    id2label = {i: label for i, label in enumerate(labels)}
    num_labels = len(labels)
    
    print(f"Entrenando modelo para clasificar en {num_labels} categorías de {label_column}:")
    for i, label in enumerate(labels):
        print(f"  {i}: {label}")
    
    # Función para tokenizar y preparar el dataset
    def tokenize_and_prepare(examples):
        # Tokenizar los textos
        tokenized = tokenizer(
            examples["sentence"], 
            padding="max_length", 
            truncation=True, 
            max_length=128
        )
        
        # Añadir las etiquetas numéricas
        tokenized["labels"] = [label2id[label] for label in examples[label_column]]
        return tokenized
    
    # Cargar y preparar los datos
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Aplicar la tokenización a los datasets
    tokenized_train = tecla_dataset["train"].map(tokenize_and_prepare, batched=True)
    tokenized_val = tecla_dataset["validation"].map(tokenize_and_prepare, batched=True)
    
    # Función para calcular métricas
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        correct = predictions == labels
        accuracy = correct.mean()
        return {"accuracy": accuracy}
    
    # Cargar modelo pre-entrenado y configurarlo para nuestra tarea
    print("Cargando el modelo base y ajustando para clasificación...")
    
    # SOLUCIÓN: Configurar explícitamente para clasificación
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True,
        problem_type="single_label_classification"  # Especificar que es clasificación
    )
    
    # Forzar el tipo de problema para asegurar que use CrossEntropyLoss
    model.config.problem_type = "single_label_classification"
    
    print(f"El modelo se ha cargado con {num_labels} etiquetas de salida")
    print(f"Tipo de problema configurado: {model.config.problem_type}")
    
    # Configuración de entrenamiento optimizada
    print("Configurando parámetros de entrenamiento...")
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=2,  # Reducido para pruebas más rápidas
        per_device_train_batch_size=16,  # Aumentado para mayor eficiencia
        per_device_eval_batch_size=16,
        warmup_steps=200,  # Reducido
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=100,
        save_steps=1000,
        eval_steps=500,  # Añadido para evaluación durante entrenamiento
        eval_strategy="steps",  # CORREGIDO: eval_strategy en lugar de evaluation_strategy
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="eval_accuracy",
        greater_is_better=True,
        do_eval=True,
        do_train=True,
        report_to=None,  # Deshabilitar wandb/tensorboard para simplificar
    )
    
    # Crear el Trainer
    print("Creando el entrenador...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,  # Añadir tokenizer
    )
    
    # Entrenar el modelo
    print("Iniciando entrenamiento del modelo...")
    trainer.train()
    print("Entrenamiento completado!")
    
    # Evaluar el modelo
    print("Evaluando el modelo...")
    eval_results = trainer.evaluate()
    print(f"Resultados de evaluación: {eval_results}")
    
    # Guardar el modelo entrenado
    model_save_path = f"./tecla_{label_column}_classifier"
    trainer.save_model(model_save_path)  # Usar trainer.save_model en lugar de model.save_pretrained
    print(f"Modelo guardado en {model_save_path}")
    
    # Ejemplos para probar la clasificación
    examples = [
        "El govern ha aprovat avui un nou decret llei per regular els preus del lloguer a les grans ciutats.",
        "La nova exposició al MNAC presenta més de 100 obres inèdites del modernisme català.",
        "L'empresa tecnològica ha anunciat la creació de 200 nous llocs de treball a Barcelona.",
        "Els estudiants han sortit al carrer per protestar contra les retallades en educació."
    ]
    
    # Clasificar los ejemplos usando el modelo directamente
    print(f"\nClasificació d'exemples en categories de {label_column}:")
    model.eval()  # Poner el modelo en modo evaluación
    
    for text in examples:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            label_id = torch.argmax(predictions, dim=-1).item()
        
        predicted_label = id2label[label_id]
        confidence = predictions[0, label_id].item()
        print(f"\nTexto: {text[:100]}...")
        print(f"Categoría predicha: {predicted_label}, Confianza: {confidence:.4f}")
    
    # Clasificar algunos ejemplos del conjunto de prueba
    print(f"\nClasificación de ejemplos del conjunto de prueba en categorías de {label_column}:")
    for i in range(5):  # Mostrar 5 ejemplos
        text = tecla_dataset["test"][i]["sentence"]
        true_label = tecla_dataset["test"][i][label_column]
        
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            label_id = torch.argmax(predictions, dim=-1).item()
        
        predicted_label = id2label[label_id]
        confidence = predictions[0, label_id].item()
        
        print(f"\nEjemplo {i+1}:")
        print(f"Texto: {text[:100]}...")
        print(f"Categoría real: {true_label}")
        print(f"Categoría predicha: {predicted_label}, Confianza: {confidence:.4f}")

except Exception as e:
    print(f"Error durante el entrenamiento o clasificación: {e}")
    import traceback
    traceback.print_exc()

Entrenando modelo para clasificar en 4 categorías de label1:
  0: Cultura
  1: Economia
  2: Política
  3: Societat
Cargando el modelo base y ajustando para clasificación...
Cargando el modelo base y ajustando para clasificación...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at projecte-aina/roberta-base-ca-v2-cased-sts and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([1, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([1]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jiahu\AppData\Local\Temp\ipykernel_16744\24078910.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
C:\Users\jiahu\AppData\Local\Temp\ipykernel_16744\24078910.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Tr

El modelo se ha cargado con 4 etiquetas de salida
Tipo de problema configurado: single_label_classification
Configurando parámetros de entrenamiento...
Creando el entrenador...
Iniciando entrenamiento del modelo...
Iniciando entrenamiento del modelo...


Step,Training Loss,Validation Loss


In [ ]:
# --- EVALUACIÓN COMPLETA DEL CONJUNTO DE VALIDACIÓN ---
print("\n" + "="*80)
print("EVALUACIÓN COMPLETA DEL CONJUNTO DE VALIDACIÓN")
print("="*80)

try:
    # Predecir para todo el conjunto de validación
    val_texts = tecla_dataset["validation"]["sentence"]
    val_true_labels = tecla_dataset["validation"][label_column]
    
    # Convertir las etiquetas reales a IDs numéricos
    val_true_label_ids = [label2id[label] for label in val_true_labels]
    
    # Inicializar listas para almacenar predicciones
    val_predictions = []
    
    # Procesar el conjunto de validación en lotes para mayor eficiencia
    batch_size = 32
    print(f"Procesando {len(val_texts)} ejemplos del conjunto de validación en lotes de {batch_size}...")
    
    for i in range(0, len(val_texts), batch_size):
        batch_texts = val_texts[i:i+batch_size]
        
        # Tokenizar los textos
        inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
        
        # Mover a la GPU si está disponible
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Realizar las predicciones
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            batch_predictions = torch.argmax(logits, dim=-1).cpu().numpy()
        
        # Añadir las predicciones a la lista
        val_predictions.extend(batch_predictions)
        
        # Mostrar progreso
        if (i // batch_size) % 10 == 0:
            print(f"Procesado hasta el ejemplo {i+len(batch_texts)}/{len(val_texts)}")
    
    # Calcular métricas
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
    
    accuracy = accuracy_score(val_true_label_ids, val_predictions)
    print(f"\nExactitud (Accuracy) en el conjunto de validación: {accuracy:.4f}")
    
    # Generar informe de clasificación
    print("\nInforme de clasificación:")
    print(classification_report(val_true_label_ids, val_predictions, target_names=labels))
    
    # Crear y mostrar matriz de confusión
    conf_matrix = confusion_matrix(val_true_label_ids, val_predictions)
    
    # Visualizar la matriz de confusión
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicción')
    plt.ylabel('Etiqueta Real')
    plt.title(f'Matriz de Confusión - Conjunto de Validación ({label_column})')
    plt.tight_layout()
    plt.show()
    
    # Analizar errores
    print("\nAnálisis de ejemplos mal clasificados:")
    
    # Encontrar ejemplos mal clasificados
    misclassified_indices = [i for i, (true, pred) in enumerate(zip(val_true_label_ids, val_predictions)) if true != pred]
    
    # Mostrar algunos ejemplos mal clasificados
    num_examples = min(5, len(misclassified_indices))
    
    if num_examples > 0:
        for i in range(num_examples):
            idx = misclassified_indices[i]
            text = val_texts[idx]
            true_label = val_true_labels[idx]
            pred_label = id2label[val_predictions[idx]]
            
            print(f"\nEjemplo mal clasificado {i+1}:")
            print(f"Texto: {text[:150]}...")
            print(f"Etiqueta real: {true_label}")
            print(f"Etiqueta predicha: {pred_label}")
            print("-"*50)
    else:
        print("¡No se encontraron ejemplos mal clasificados!")
    
    # Guardar métricas y resultados
    results = {
        "accuracy": accuracy,
        "confusion_matrix": conf_matrix.tolist(),
        "num_examples": len(val_texts),
        "num_misclassified": len(misclassified_indices)
    }
    
    import json
    with open(f"validation_metrics_{label_column}.json", "w") as f:
        json.dump(results, f, indent=2)    
    print(f"\nLos resultados se han guardado en 'validation_metrics_{label_column}.json'")
    
except Exception as e:
    print(f"Error durante la evaluación: {e}")
    import traceback
    traceback.print_exc()


EVALUACIÓN COMPLETA DEL CONJUNTO DE VALIDACIÓN
Procesando 5669 ejemplos del conjunto de validación en lotes de 32...
Procesado hasta el ejemplo 32/5669
Procesado hasta el ejemplo 32/5669


KeyboardInterrupt: 

## Resultados de Evaluación

Se ha evaluado el modelo entrenado en el conjunto de validación completo. Los resultados principales incluyen:

1. **Exactitud (Accuracy)**: Porcentaje de ejemplos correctamente clasificados
2. **Informe de clasificación**: Métricas de precisión, recall y F1-score para cada categoría
3. **Matriz de confusión**: Visualización de predicciones correctas vs incorrectas
4. **Análisis de errores**: Ejemplos de textos mal clasificados para identificar limitaciones del modelo

Estas métricas permiten evaluar la calidad y fiabilidad del modelo para la tarea de clasificación en catalán.

In [ ]:
# Visualización adicional de resultados

# Cargar los resultados si existen
import json
import os
import matplotlib.pyplot as plt
import numpy as np

result_file = f"validation_metrics_{target_label}.json"

if os.path.exists(result_file):
    with open(result_file, "r") as f:
        results = json.load(f)
    
    # Mostrar un resumen visual de los resultados
    print(f"Resumen de evaluación para clasificación de {target_label}:")
    print(f"Exactitud (Accuracy): {results['accuracy']:.4f}")
    print(f"Total de ejemplos evaluados: {results['num_examples']}")
    print(f"Ejemplos mal clasificados: {results['num_misclassified']}")
    
    # Crear un gráfico de barras para la exactitud por clase
    # Esto requeriría datos adicionales, pero podemos hacer una visualización simple
    if 'confusion_matrix' in results:
        conf_matrix = np.array(results['confusion_matrix'])
        class_accuracy = np.diag(conf_matrix) / conf_matrix.sum(axis=1)
        
        plt.figure(figsize=(10, 6))
        plt.bar(labels, class_accuracy, color='skyblue')
        plt.axhline(y=results['accuracy'], color='r', linestyle='-', label=f'Exactitud global: {results["accuracy"]:.4f}')
        plt.xlabel('Categoría')
        plt.ylabel('Exactitud')
        plt.title(f'Exactitud por Categoría - {target_label}')
        plt.xticks(rotation=45)
        plt.ylim(0, 1.0)
        plt.legend()
        plt.tight_layout()
        plt.show()
else:
    print(f"No se encontró el archivo de resultados: {result_file}")
    print("Ejecuta primero la celda de evaluación del conjunto de validación.")